In [1]:
# Run first in every notebook
import os, sys, pathlib
if pathlib.Path().resolve().name == "notebooks":
    os.chdir("..")                         # move CWD to project root
sys.path.insert(0, os.path.abspath("src")) # make `src/` importable
print("CWD:", os.getcwd())

CWD: /Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis


In [2]:
import sys; print(sys.executable)


/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/.venv/bin/python


In [3]:
from src.fetch_data import load_cleaned_trials
from src.bayesian_posterior import fit_models_by_participant

df = load_cleaned_trials("data/exports/trials_20250811T043446Z_cleaned.csv")
fits = fit_models_by_participant(df)
fits

,participantId,n,m1_nll,m1_aic,m1_bic,m1_kappa,m1_bias,m1_lapse,m2_nll,m2_aic,m2_bic,m2_alpha,m2_beta,m2_kappa,m2_bias,m2_lapse,dAIC,dBIC
0,"(44ab8d9a-01ff-4cb3-8676-c933a77b7207,)",445,87.945268,181.890535,194.184758,14.018485,-0.296418,0.000000,84.414270,178.828541,199.318912,0.871740,0.822841,15.122104,-0.947032,0.0,3.061995,-5.134154
1,"(696e0e3f-da68-4178-8d5b-43ea5aaefadd,)",445,144.512040,295.024080,307.318303,7.656729,0.001113,0.000000,139.889874,289.779748,310.270120,0.704488,0.903451,8.438204,-0.042536,0.0,5.244331,-2.951817
2,"(7a85d59e-1e5c-4d1d-8dc1-5c866967a30f,)",445,139.346292,284.692585,296.986808,8.018862,0.170290,0.000000,129.295119,268.590238,289.080609,0.792101,0.600161,9.571984,-0.954614,0.0,16.102347,7.906199
3,"(abbf0f48-70cd-4589-8b1f-6570cee03296,)",445,176.473361,358.946723,371.240946,5.764074,0.035259,0.007562,165.225614,340.451228,360.941599,0.515345,0.780760,6.937296,-0.143841,0.0,18.495495,10.299347


In [4]:
from src.plots import weighting_curves
weighting_curves(fits)

# Model comparison summary and exports
import numpy as np
fits["better_by_AIC"] = np.where(fits["dAIC"] > 0, "Prelec", "Unweighted")
fits["better_by_BIC"] = np.where(fits["dBIC"] > 0, "Prelec", "Unweighted")
fits.to_csv("data/derived/model_fits.csv", index=False)
fits[["participantId","dAIC","dBIC","m2_alpha","m2_beta","m2_kappa","m2_bias","m2_lapse"]]

,participantId,dAIC,dBIC,m2_alpha,m2_beta,m2_kappa,m2_bias,m2_lapse
0,"(44ab8d9a-01ff-4cb3-8676-c933a77b7207,)",3.061995,-5.134154,0.871740,0.822841,15.122104,-0.947032,0.0
1,"(696e0e3f-da68-4178-8d5b-43ea5aaefadd,)",5.244331,-2.951817,0.704488,0.903451,8.438204,-0.042536,0.0
2,"(7a85d59e-1e5c-4d1d-8dc1-5c866967a30f,)",16.102347,7.906199,0.792101,0.600161,9.571984,-0.954614,0.0
3,"(abbf0f48-70cd-4589-8b1f-6570cee03296,)",18.495495,10.299347,0.515345,0.780760,6.937296,-0.143841,0.0


In [5]:
import numpy as np, pandas as pd
from src.compute_likelihood import prelec_weight

rows = []
for _, r in fits.iterrows():
    alpha, beta = r["m2_alpha"], r["m2_beta"]
    p = np.linspace(0.001, 0.999, 99)
    w = prelec_weight(p, alpha, beta)
    rows.append(pd.DataFrame({"participantId":[r["participantId"]]*len(p), "p":p, "w":w}))
pd.concat(rows).head()

,participantId,p,w
0,"(44ab8d9a-01ff-4cb3-8676-c933a77b7207,)",0.001000,0.011842
1,"(44ab8d9a-01ff-4cb3-8676-c933a77b7207,)",0.011184,0.047398
2,"(44ab8d9a-01ff-4cb3-8676-c933a77b7207,)",0.021367,0.069777
3,"(44ab8d9a-01ff-4cb3-8676-c933a77b7207,)",0.031551,0.088420
4,"(44ab8d9a-01ff-4cb3-8676-c933a77b7207,)",0.041735,0.105021
